In [ ]:
import numpy as np
import skimage
import skimage.filters
import nd2reader
import holoviews as hv
import matplotlib.pyplot as plt
import numba
import trench_detection, ui, diagnostics, image

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
hv.extension("bokeh")

In [ ]:
nd2 = nd2reader.ND2Reader("/n/scratch2/jqs1/fidelity/all/180405_txnerr.nd2")

# Hough averaging

In [ ]:
def diff_profile(h):
    return np.diff(h.astype(np.int64), axis=1).std(axis=0)

In [ ]:
%%time
imgs = [np.asarray(nd2.get_frame_2D(t=0, c=2, v=v)) for v in range(80)]

In [ ]:
theta = np.linspace(np.deg2rad(-10), np.deg2rad(10), 200)

In [ ]:
%%time
houghs = [image.hough_line_intensity(img, theta=theta)[0] for img in imgs]

In [ ]:
%%time
diff_h_stds = [diff_profile(h) for h in houghs]

In [ ]:
hv.Overlay.from_values(
    [
        hv.Curve((np.rad2deg(theta[:-1]), diff_h_std / diff_h_std.max()))
        for diff_h_std in diff_h_stds
    ]
)

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(np.rad2deg(theta[:-1]), np.array(diff_h_stds).T)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(houghs[0], aspect=0.02)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(houghs[-1], aspect=0.02)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(houghs[-2].astype(np.int64) - houghs[0], aspect=0.02)

# Old

In [ ]:
img = np.asarray(nd2.get_frame_2D(t=0, c=2, v=50))

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(img)

In [ ]:
plt.imshow(img[: img.shape[0] // 2, :])

In [ ]:
plt.imshow(img[img.shape[0] // 2 :, :])

In [ ]:
theta = np.linspace(np.deg2rad(-10), np.deg2rad(10), 200)

In [ ]:
h0, _, _ = image.hough_line_intensity(img, theta=theta)

In [ ]:
h, _, _ = image.hough_line_intensity(img[img.shape[0] // 2 :, :], theta=theta)

In [ ]:
h2, _, _ = image.hough_line_intensity(img[: img.shape[0] // 2, :], theta=theta)

In [ ]:
%%output size=200
hv.HoloMap({i: hv.Image(d) for i, d in enumerate([h0, h, h2])})

In [ ]:
plt.plot(diff_profile(h0))
plt.plot(diff_profile(h))
plt.plot(diff_profile(h2))

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(h0[2000:, 80:100], aspect=0.01)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(h[2000:, 80:100], aspect=0.01)

In [ ]:
trenches, diag, _ = trench_detection.get_trenches_diagnostics(img)

In [ ]:
ui.display_plot_browser(diag)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(bin_img)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(h4[3000:3200, 80:100], aspect=0.1)

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(h[3000:3200, 90] / h.max())
plt.plot(h4[3000:3200, 90] / h4.max())

In [ ]:
h3

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow((h3.astype(np.int64) - h2), aspect=0.02)

In [ ]:
h0, _, _ = skimage.transform.hough_line(
    bin_img, theta=np.deg2rad(np.linspace(-10, 10, 100))
)

In [ ]:
h5, _, _ = image.hough_line_intensity(img, theta=np.deg2rad(np.linspace(-10, 10, 100)))

In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(h0[2500:5000, :], aspect=0.05)

In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(h5[2500:5000, :], aspect=0.05)

# New

In [ ]:
img = np.asarray(nd2.get_frame_2D(t=0, c=2, v=30))

In [ ]:
plt.imshow(img)

In [ ]:
trench_points, diag, e = trench_detection.get_trenches_diagnostics(img)

In [ ]:
ui.display_plot_browser(diag)